# Custom Dataset Practice
- Manual transforms
- Custom Dataset
- Custom 

In [ ]:
%matplotlib inline
import os

import torch
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, models

from torchsummary import summary

device = torch.device('cuda')

In [ ]:
# fileid: 0BxYys69jI14kRjNmM0gyVWM2bHM
# filename: crop_part1.tar.gz

!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=0BxYys69jI14kRjNmM0gyVWM2bHM' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=0BxYys69jI14kRjNmM0gyVWM2bHM" -O crop_part1.tar.gz && rm -rf ~/cookies.txt
!tar -zxvf crop_part1.tar.gz

In [ ]:
# files list 생성 


In [ ]:
# Data 이상 없는지 확인

In [ ]:
# file path preprocess
fname = '1_1_0_20161219204750596.jpg.chip.jpg'


In [ ]:
img = Image.open(
    './crop_part1/1_1_0_20161219204750596.jpg.chip.jpg').convert('RGB')
img

### Torchvision transforms

In [ ]:
# torchvision.transforms 적용해보기
transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

In [ ]:
# Implementation RandomChoice of 
#       [ Grayscale(),
#        CenterCrop(100),
#        ColorJitter(0, 0, 0, 0),
#        GaussianBlur(kernel_size=9, sigma=(0.1, 10.0)),
#        ],
#        Resize back to (200,200)

tr = 

tr(img)

In [ ]:
# Implementation RandomOrder of 
#       [ 
#        ColorJitter(0, 0, 0, 0),
#        RandomAffine((0,360),translate=(0.3, 0.4), scale = (0.8,1.2), shear=(-30,30))
#        ],
#        Resize back to (200,200)

tr = 

tr(img)

### Custom Dataset
다음의 세가지 attribute는 반드시 정의
- __init__(self, *args, **kargs)
- __len__(self)
- __getitem__(self, idx)

In [ ]:
class FaceDataset(Dataset):
    def __init__(self, root_dir, transform):
        super().__init__()
        # implement this.
        # 필요한 정보들 생성.
        # Hint: os.listdir(directory) -> List[filename]

        
    def __len__(self):
        # implement this.
        # 데이터 셋의 크기가 얼마인지? (또는, image가 몇장 있는지)
        return 
        
    def __getitem__(self, idx):
        # implement this
        # 인덱스 (idx)를 받아서, 해당 idx 의 data 와 label 을 반환.
        # Hint: Image.open(filename).convert('RGB')
        # Hint: '1' -> 1. 로 바꾸기 위해선 float() 을 사용.
        # Hint: filename = {age}_{gender}_{race}_{time}.jpg
        return {
            "image": img, # [3, 224, 224]
            "age": age,
            "gender": gender,
            "race": race,
            "filename": filename
        }

In [ ]:
# implement dataset and dataloader, train, test split maybe val set
dataset = FaceDataset("./crop_part1", transform)
train_length = int(len(dataset) * 0.9)
test_length = len(dataset) - train_length
train_set, test_set = torch.utils.data.random_split(dataset, [train_length, test_length])
print(len(train_set), len(test_set))

### Custom collate function

In [ ]:
def my_collate_fn(list_data):
#     [dataset.__getitem__(0), ..., dataset.__getitem__(7)]

    print("My collate function is working!!!!")
    data_dicts = {
        "image": [], # torch tensor [8, 3, 512, 512]
        "age": [], # torch tensor [8]
        "gender": [], # torch tensor [8]
        "race": [], # torch tensor [8]
        "filename": [] # torch tensor [8]
    }
    
    batch_size = len(list_data)
    for i in range(batch_size):
        data = list_data[i] # {"image": torch.Tensor, "age": float, "filename": str}
        data_dicts["image"].append(data["image"].unsqueeze(0)) # [1, 3, 224, 224]
        data_dicts["age"].append(data["age"])
        data_dicts["gender"].append(data["gender"])
        data_dicts["race"].append(data["race"])
        data_dicts["filename"].append(data["filename"])
    data_dicts["image"] = torch.cat(data_dicts["image"], dim=0)
    data_dicts["age"] = torch.tensor(data_dicts["age"])
    data_dicts["gender"] = torch.tensor(data_dicts["gender"])
    data_dicts["race"] = torch.tensor(data_dicts["race"])
    return data_dicts

In [ ]:
batch_size = 8
trainloader = DataLoader(
    train_set, batch_size = batch_size, shuffle = True, collate_fn=my_collate_fn
)
testloader = DataLoader(
    test_set, batch_size = batch_size, shuffle = False, collate_fn=my_collate_fn
)

In [ ]:
# display some images

import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img / 2 + 0.5  # Unnormalize
    npimg = img.numpy()
    plt.figure(figsize=(20,12))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
    
# get some random training images
dataiter = iter(trainloader)
data_dicts = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(data_dicts["image"]))

# print labels
Gender_dict = {0 : 'Male', 1 : 'Female'}
Race_dict = {0 : 'White', 1 : 'Black', 2 : 'Asian', 3: 'Indian', 4 : 'Others'}
print('Age    : ', ' '.join('%6s' % str(data_dicts["age"][j].item()) for j in range(8)))
print('Gender : ', ' '.join('%6s' % Gender_dict[int(data_dicts["gender"][j].item())] for j in range(8)))
print('Race   : ',' '.join('%6s' % Race_dict[int(data_dicts["race"][j].item())] for j in range(8)))

---
## Age Regression model

In [ ]:
# ResNet18 pretrained network 받아서 씀
class Net(nn.Module):
    def __init__(self, use_pretrained=True):
        super().__init__()
        self.net = models.resnet18(pretrained = use_pretrained) #fc layer 3 는 빼고 가져옴
        in_features = self.net.fc.in_features
        self.net.fc = nn.Linear(in_features, 1) #512, 1
        
    def forward(self, x):
        return self.net(x)

In [ ]:
criterion = nn.MSELoss()

model = Net(use_pretrained=True)
summary(model, batch_size=-1, input_size=(3, 224, 224), device='cpu')
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
def train(epoch):
    model.train()
    total_loss = 0.
    for batch_idx, data_dict in enumerate(trainloader):
        data = data_dict["image"].to(device)
        target = data_dict["age"].to(dtype=torch.float32, device = device).view(-1,1)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), total_loss/(batch_idx+1)))

def test():
    model.eval()
    with torch.no_grad():
        test_loss = 0
        correct = np.zeros(1)
        total = np.zeros(1)
        for data_dict in testloader:
            target = data_dict["age"].to(dtype=torch.float32, device = device).view(-1,1)
            
            data, target = data_dict["image"].to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            
            correct += np.sum(np.array(target.to('cpu')) == np.round(np.array(output.to('cpu'))), 0)
            total += len(target)
            
        acc = correct/total
        test_loss /= len(testloader.dataset)
        print(f'\nTest set: Average loss: {test_loss:.4f}',
                f'Correct Labels for Ages : {acc[0]:.3f}')

In [ ]:
for epoch in range(5):
    train(epoch)
    test()

In [ ]:
dataiter = iter(testloader)
data_dict = dataiter.next()
images = data_dict['image']

imshow(torchvision.utils.make_grid(images))
print('GrondTruth: ', ' '.join('%5s' % data_dict['age'][j].item() for j in range(8)))

In [ ]:
images = images.to(device)
outputs = model(images)

outputs = outputs.squeeze()

print('Predicted: ', ' '.join('%.1f' % outputs[j].item() for j in range(8))) 

# Implement Regression model for Age / Gender / Race at once

In [ ]:
class Net(nn.Module):
    

In [ ]:
model = Net(use_pretrained=True)
summary(model, batch_size=-1, input_size=(3, 224, 224), device='cpu')
model = model.to(device)

criterion = 
optimizer = 

In [ ]:
def train(epoch):
    model.train()
    total_loss = 0.
    for batch_idx, data_dict in enumerate(trainloader):
        # Implement
        data = 
        label = 
        
        
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), total_loss/(batch_idx+1)))
           

def test():
    model.eval()
    with torch.no_grad():
        test_loss = 0
        correct = np.zeros(3)
        total = np.zeros(3)
        for data_dict in testloader:
            data = 
            label = 
            output = model(data)
            
            test_loss += criterion(output, label).item()
            
            correct += 
            total += len(target)
            
        acc = correct/total
        test_loss /= len(testloader.dataset)
        print(f'\nTest set: Average loss: {test_loss:.4f}',
                f'Correct Labels for Ages : {acc[0]:.3f}, Gender : {acc[1]:.3f}, Race : {acc[1]:.3f}')

In [ ]:
dataiter = iter(testloader)
data_dict = dataiter.next()
images = data_dicts['image']

imshow(torchvision.utils.make_grid(images))
print('Age    : ', ' '.join('%6s' % str(data_dicts["age"][j].item()) for j in range(8)))
print('Gender : ', ' '.join('%6s' % Gender_dict[int(data_dicts["gender"][j].item())] for j in range(8)))
print('Race   : ',' '.join('%6s' % Race_dict[int(data_dicts["race"][j].item())] for j in range(8)))

In [ ]:
images = images.to(device)
outputs = model(images)
outputs = np.array(outputs.detach().to('cpu'))
pred_ages = outputs[:,0]
pred_gender = ((outputs[:, 1])>0.5).astype(int)
race = outputs[:,2]
for idx in range(len(race)):
    val = race[idx]
    if val <= 0.5:
        race[idx] = 0
    elif val > 3.5:
        race[idx] = 4
    else:
        race[idx] = np.round(val)
pred_race = race.astype(int)

In [ ]:
print('Age    : ', ' '.join('%6.1f' % pred_ages[j] for j in range(8)))
print('Gender : ', ' '.join('%6s' % Gender_dict[pred_gender[j]] for j in range(8)))
print('Race   : ',' '.join('%6s' % Race_dict[pred_race[j]] for j in range(8)))